## Linear Regression


Sklearn based models are slightly finicky to get into a suitable onnx format. 
This notebook showcases how to do so using the `hummingbird-ml` python package ! 

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hummingbird-ml"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import torch
import ezkl
import json
from hummingbird.ml import convert


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

circuit = convert(reg, "torch", X[:1]).model

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('model.compiled')
pk_path = os.path.join('pk.key')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
# read in ./input_json
data = json.load(open("input.json", 'r'))
# convert to torch tensor
x = torch.tensor(data['input_data'], requires_grad=True)
torch_out = circuit(x)
# Export the model
torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

In [ ]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources", scales=[4,7])
assert res == True

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs(settings_path)

In [ ]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
import subprocess
import re
import os
import platform


if platform.system() == 'Linux':
    time_command = "/usr/bin/time"
else:
    time_command = "gtime"

# Determine the correct time command based on the OS

# Define the command to run with GNU time
gnu_time_command = [time_command, "-f", "%M", "--"]
ezkl_command = ["ezkl", "prove", "--check-mode=UNSAFE"]
full_command = gnu_time_command + ezkl_command

# Run the command and capture output
result = subprocess.run(full_command, capture_output=True, text=True)

# Extract the outputs
output = result.stdout
error_output = result.stderr

# Use regular expression to find the "proof took" value
proof_time_match = re.search(r"proof took (\d+\.\d+)", output)
proof_time = proof_time_match.group(1) if proof_time_match else None

# Use regular expression to find the memory usage value from GNU time
memory_usage_match = re.search(r"(\d+)", error_output)
memory_usage = memory_usage_match.group(1) if memory_usage_match else None

# Display the results
print(f"Proof Time: {proof_time} seconds")
print(f"Memory Usage: {memory_usage} KB")


# define the path that stores the benchmarking results
benchmark_path = os.path.join('../../benchmarks.json')

# assume benchmark file has already been created (run `bash benchmark_file.sh` to create it)
with open(benchmark_path, 'r') as f:
    benchmark = json.load(f)

proving_time =str(proof_time) + "s"

memory_used = str(memory_usage) + "kb"

# Update the proving time in the loaded benchmark
benchmark['linear_regression']['ezkl']['provingTime'].append(proving_time)

# Update the memory usage in the loaded benchmark
benchmark['linear_regression']['ezkl']['memoryUsage'].append(memory_used)

# Write the updated benchmark back to the file
with open(benchmark_path, 'w') as f:
    json.dump(benchmark, f, indent=4)

#assert proof path exists
assert os.path.isfile('proof.json')